# Network Deliberation ABM
## Discrete state, generated NK Model task

In [ ]:
# Imports
import configparser
import math

from matplotlib import pyplot as plt
import netdelib.soclearn as slearn
import numpy as np
import pandas as pd
import repsci

In [ ]:
# Configuration
reproduce = '2020-11-18 164248 d441939'
experiment = repsci.Experiment(
    "discrete_generated_nk",
    reproduce=reproduce)
config = experiment.get_config()

N = config.getint('abm', 'N')
M = config.getint('abm', 'M')
runs = config.getint('abm', 'runs')
stages = config.getint('abm', 'stages')
steps = config.getint('abm', 'steps')

bit_count = config.getint('abm', 'bit_count')
K = config.getint('abm', 'K')
p_error = config.getfloat('abm', 'p_error')
nk_exponent = config.getfloat('abm', 'nk_exponent', fallback=1)

small_world_k = config.getint('networks', 'small_world_k')
small_world_a = config.getfloat('networks', 'small_world_a')
barabasi_albert_m = config.getint('networks', 'barabasi_albert_m')
er_p = (M - 1)/(N - 1)

# Configure plotting in Jupyter
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams.update({
    'figure.figsize': (20, 20)})

# Configure Networks
all_networks = [
    'Complete',
    'Lattice',
    'Pref. Attach.',
    'Small World',
    'LFR',
    'StochasticBlock',
    'Random',
    'Long Path',
    'Random Group',
]
networks = config.get('networks', 'enabled').split(',')

# Configure Strategies
individual = config.getboolean('strategies', 'individual', fallback=False)
individual_mode = config.getint('strategies', 'individual_mode', fallback=1)
individual_all_bits = config.getboolean('strategies', 'individual_all_bits', fallback=True)
critical = config.getboolean('strategies', 'critical')
sample = config.getint('strategies', 'sample', fallback=None)
all_learning_strategies = [
    'Best Neighbor',
    'Conform',
    'Local Majority',
]
learning_strategies = config.get('strategies', 'enabled').split(',')

num_net = len(networks)
num_strat = len(learning_strategies)

In [ ]:
score = np.load(experiment.get_filename('score.npy'))
correct = np.load(experiment.get_filename('correct.npy'))
distance_score = np.load(experiment.get_filename('distance_score.npy'))

In [ ]:
symbols = ['o','d','s']
colors = ['#7f3f3f', '#3f7f3f', '#3f3fff']
symbols = ['.','.','.']
colors = ['#3f3fff','#3f3fff','#3f3fff']
plt.figure(figsize=(7,5.5))
exclude = ['Random', 'StochasticBlock']

if individual:
    if individual_all_bits:
        title_individual = 'all-bit'
    else:
        title_individual = '1-bit'
    if individual_mode == slearn.MODE_ALL:
        title_individual += ' every-step indiv.'
    elif individual_mode == slearn.MODE_FALLBACK:
        title_individual += ' fallback-only indiv.'
    elif individual_mode == slearn.MODE_BEST:
        title_individual += ' hybrid indiv.'
else:
    title_individual = 'social-only'
title = "|V|={} N={}, K={}, EXP={}, S={}, {}, {}".format(
    N,
    bit_count,
    K,
    nk_exponent,
    sample,
    {True: 'critical', False: 'non-critical'}[critical],
    title_individual)

for i, title in enumerate(learning_strategies):
    plt.subplot(1,len(learning_strategies),1+i)
    xlabels = []
    ys = []
    err95s = []
    for j, title2 in enumerate(networks):
        if title2 in exclude:
            continue
        xlabels.append(title2)
        y = score[i][j][0]
        best = max(y)
        ys.append(best)
        converge_threshold = 0.99 * best
        converge_time = None
        for t, y_t in enumerate(y):
            if y_t >= converge_threshold:
                converge_time = t
                break
        err95 = 1.96 * np.array(score[i][j][1]) / math.sqrt(runs)
        err95s.append(err95[t])
    plt.title(title, fontsize=16)
    plt.errorbar(range(len(xlabels)), ys, yerr=err95s, fmt='{}'.format(symbols[i]), markerfacecolor='{}ff'.format(colors[i]), color=colors[i], markeredgecolor='{}ff'.format(colors[i]), label=title, markersize=12)
    plt.ylim([0, 1])
    plt.grid(True)
    plt.xticks(ticks=range(len(xlabels)), labels=xlabels, rotation=90, fontsize=16)
    plt.yticks(fontsize=16)
    if i == 0:
        plt.ylabel('Mean Final Score', fontsize=20)

#plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1), fontsize=12)
plt.tight_layout()
out = experiment.get_filename('performance.png')
plt.savefig(out, dpi=600)
plt.savefig(experiment.get_filename('performance.eps'), dpi=600)


In [ ]:
plt.figure(figsize=(30, 18))
for i, title in enumerate(learning_strategies):
    for j, title2 in enumerate(networks):
        plt.subplot(num_strat, num_net, 1 + i * num_net + j)

        y = score[i][j][0]
        x = range(len(y))
        err95 = 1.96 * np.array(score[i][j][1]) / math.sqrt(runs)
        plt.fill_between(x, y - err95, y + err95, color="#aaaaff7f")
        plt.plot(x, y, 'k-')     

        y = correct[i][j][0]
        x = range(len(y))
        err95 = 1.96 * np.array(correct[i][j][1]) / math.sqrt(runs)
        plt.fill_between(x, y - err95, y + err95, color="#aaaaff7f")
        plt.plot(x, y, 'b:')
        
        y = distance_score[i][j][0]
        x = range(len(y))
        err95 = 1.96 * np.array(distance_score[i][j][1]) / math.sqrt(runs)
        plt.fill_between(x, y - err95, y + err95, color="#aaaaff7f")
        plt.plot(x, y, 'g--')
        
        plt.title(f'{title} - {title2}')
        if j == 0:
            plt.ylabel('Frac. Correct')
        if i == 2:
            plt.xlabel('Time Step')
        plt.grid(True)
        plt.tight_layout()
        plt.ylim([0.0, 1])

out = experiment.get_filename('plot.png')
plt.savefig(out, dpi=300)